<a href="https://colab.research.google.com/github/guebin/DL2025/blob/main/posts/13wk-1.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" style="text-align: left"></a>

# 1. 강의영상 

{{<video https://youtu.be/playlist?list=PLQqh36zP38-xkKWgGZTE_11TnXRdBtAW-&si=dMVGdO_bdkObYiLO >}}

# 2. Imports 

In [190]:
import numpy as np
import collections

# 3. 강화학습 Intro 

`-` 강화학습(대충설명): 어떠한 "(게임)환경"이 있을때 거기서 "뭘 할지"를 학습하는 과업

![그림1: 셔튼(@sutton1998reinforcement)의 교재에서 발췌한 그림, 되게 유명한 그림이에요](https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig1.png?raw=true)

`-` 딥마인드: breakout $\to$ 알파고 

- <https://www.youtube.com/watch?v=TmPfTpjtdgg>

![그림2: 벽돌깨기](https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig2.png?raw=true)

`-` 강화학습에서 "강화"는 뭘 강화한다는것일까? 

- <https://k9connoisseur.com/blogs/news/positive-reinforcement-dog-training> 

`-` 강화학습 미래? (이거 잘하면 먹고 살 수 있을까?) 

# 4. Bandit 게임 설명

`-` 문제설명: 두 개의 버튼이 있다. `버튼0`을 누르면 1의 보상을, `버튼1`을 누르면 10의 보상을 준다고 가정 

- Agent: 버튼0을 누르거나,버튼1을 누르는 존재 
- Env: Agent의 Action을 바탕으로 Reward를 주는 존재

> 주의: 이 문제 상황에서 state는 없음 

`-` 생성형AI로 위의 상황을 설명한것 (왼쪽부터 챗지피티, 제미나이, 퍼플렉시티의 결과)

::: {layout-ncol=3}
![](https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-gpt.png?raw=true)

![](https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-gemini.png?raw=true)

![](https://github.com/guebin/DL2025/blob/main/posts/13wk-1-fig3-perplexity.png?raw=true)

:::

- 클로드로 생성: <https://claude.ai/public/artifacts/1f52fcb2-ef08-4af1-8cf8-4a497d7bcc5f>

`-` 게임진행양상

- 처음에는 아는게 없음. 일단 "아무거나" 눌러보자. ("에이전트가 랜덤액션을 한다" 고 표현함 )
- 한 20번 정도 눌러보면서 결과를 관찰함 ("에이전트가 경험을 축적한다"고 표현함) 
- 버튼0을 누를때는 1점, 버튼1을 누를때는 10점을 준다는 사실을 깨달음. ("에이전트가 환경을 이해했다"고 표현함)
- 버튼1을 누르는게 나한테 이득이 라는 사실을 깨달음. ("에이전트가 최적의 정책을 학습했다" 고 표현함)
- 이제부터 무조건 버튼1만 누름 $\to$ 게임 클리어 ("강화학습 성공"이라 표현할 수 있음)

`-` 어떻게 버튼1을 누르는게 이득이라는 사실을 아는거지? $\to$ 아래와 같은 테이블을 만들면 된다. (`q_table`)

|| Action0 | Action1 | 
|:-:|:-:|:-:|
|State0 | mean(Reward \| State0, Action0) |mean(Reward \| State0, Action1)|

# 5.  Bandit 환경 설계 및 풀이  

## A. 대충 개념만 실습

In [517]:
action_space = [0,1] 
actions_deque = collections.deque(maxlen=500)
rewards_deque =  collections.deque(maxlen=500)
#---#

In [518]:
for _ in range(10):
    action = np.random.choice(action_space)
    if action == 1:
        reward = 10 
    else:
        reward = 1
    actions_deque.append(action)
    rewards_deque.append(reward)

In [519]:
actions_deque

deque([0, 1, 0, 0, 0, 1, 0, 1, 1, 0], maxlen=500)

In [520]:
rewards_deque

deque([1, 10, 1, 1, 1, 10, 1, 10, 10, 1], maxlen=500)

In [521]:
actions_numpy = np.array(actions_deque)
rewards_numpy = np.array(rewards_deque)

In [522]:
q0 = rewards_numpy[actions_numpy == 0].mean()
q1 = rewards_numpy[actions_numpy == 1].mean()
q_table = np.array([q0,q1])
q_table

array([ 1., 10.])

In [523]:
action = q_table.argmax()

In [524]:
for _ in range(5):
    action = q_table.argmax()
    if action == 1:
        reward = 10 
    else:
        reward = 1
    actions_deque.append(action)
    rewards_deque.append(reward)
    actions_numpy = np.array(actions_deque)
    rewards_numpy = np.array(rewards_deque)    
    q0 = rewards_numpy[actions_numpy == 0].mean()
    q1 = rewards_numpy[actions_numpy == 1].mean()
    q_table = np.array([q0,q1])

In [526]:
actions_numpy

array([0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1])

In [525]:
rewards_numpy

array([ 1, 10,  1,  1,  1, 10,  1, 10, 10,  1, 10, 10, 10, 10, 10])

## B. 클래스를 이용한 구현 

In [533]:
class Bandit:
    def __init__(self):
        self.reward = None 
    def step(self,action):
        if action == 0:
            self.reward = 1
        else: 
            self.reward = 10 
        return self.reward 

In [534]:
env = Bandit()

In [540]:
class Agent:
    def __init__(self):
        pass 
    def act(self):
        # 만약에 경험이 20보다 작음 --> 랜덤액션 
        # 경험이 20보다 크면 --> action = q_tabel.argmax()
        pass 
    def save_experience(self):
        # 데이터 저장 
        pass 
    def learn(self):
        # q_table 을 업데이트하는 과정 
        pass

:::{.callout-important}
앞으로의 수업에서는 아래에 해당하는 클래스의 기본 개념을 숙지하셔야 합니다. (`13wk-2` 주차 강의듣기전까지 꼭!) 

1. 클래스와 인스턴스의 개념, `__init__`, `self`, 메소드
2. 클래스의 상속 

관련하여 제가 작년에 수업한 자료는 아래와 같습니다

1. <https://guebin.github.io/PP2024/posts/11wk-2.html> 에서 1-7까지.. 
2. <https://guebin.github.io/PP2024/posts/14wk-2.html> 에서 8-A

물론, 꼭 제 강의노트로만 공부하셔야하는것은 아닙니다. 제 수업 외에도 클래스를 잘 설명하는 다양한 자료들이 많이 있으니 자유롭게 참고하여 학습하시기 바랍니다.
:::